# Notebook as a Step using Notebooks Executor
The following sample shows how to use the notebook executor as part of a Vertex AI


## Libraries and Variables

In [1]:
!which pip

/Users/matthieum/development/miniconda/envs/naas/bin/pip


In [2]:
!pip install --upgrade google-cloud-aiplatform -q
!pip install --upgrade google-cloud-pipeline-components -q

In [2]:
import kfp
import os

from datetime import datetime
from google.cloud import aiplatform
from google.cloud.aiplatform import pipeline_jobs
from kfp.v2 import compiler

In [3]:
kfp.__version__

'1.8.2'

In [4]:
# Variables
PROJECT_ID = 'mam-nooage'
REGION = 'us-central1' # This is where the KMS and Metastore resides from the project configuration

ROOT_PATH = "."
PIPELINE_ROOT_PATH = f'gs://{PROJECT_ID}-vertex-root'
PACKAGE_PATH = 'notebook-as-a-step-sample-pipeline.json'

RUNNABLES_PATH = './runnables'
COMPONENT_YAML_PATH = os.path.join(ROOT_PATH, 'component.yaml')
if not os.path.isfile(COMPONENT_YAML_PATH):
    print(f'COMPONENT_YAML_PATH does not exist')

WORKING_BUCKET_NAME = f'{PROJECT_ID}-naas'
INPUT_NOTEBOOK_FILE = f'gs://{WORKING_BUCKET_NAME}/runnables/run_create_bucket.ipynb'
OUTPUT_NOTEBOOK_FOLDER = f'gs://{WORKING_BUCKET_NAME}/outputs'

INPUT_NOTEBOOK_FILE

'gs://mam-nooage-naas/runnables/run_create_bucket.ipynb'

In [5]:
!gcloud config set project "{PROJECT_ID}"
!gcloud config list

Updated property [core/project].
[billing]
quota_project = mam-nooage
[compute]
region = us-central1
zone = us-central1-b
[core]
account = matthieum@google.com
disable_usage_reporting = False
project = mam-nooage

Your active configuration is: [staging-alpha]


In [6]:
# The service account used by the Pipelines must have access to this bucket.
!gsutil ls "{PIPELINE_ROOT_PATH}" || gsutil mb -l "{REGION}" "{PIPELINE_ROOT_PATH}"
!gsutil cp -r "{RUNNABLES_PATH}/*" "gs://{WORKING_BUCKET_NAME}/runnables"
!gsutil ls "gs://{WORKING_BUCKET_NAME}/runnables"

gs://mam-nooage-vertex-root/780283066030/
CommandException: No URLs matched: ./runnables/*
gs://mam-nooage-naas/runnables/parameters.yaml
gs://mam-nooage-naas/runnables/run_create_bucket.ipynb


## Prerequisites
You need to configure your project as detailed in https://cloud.google.com/vertex-ai/docs/pipelines/configure-project

In [7]:
@kfp.dsl.pipeline(
    name="notebook-as-a-step-sample",
    pipeline_root=PIPELINE_ROOT_PATH)
def pipeline(
        project_id: str, 
        execution_id: str, 
        input_notebook:str, 
        output_notebook_folder:str):
    
    execute_notebook_component = kfp.components.load_component_from_file(COMPONENT_YAML_PATH)
    execute_notebook_op = execute_notebook_component(
        project=project_id,
        execution_id=execution_id,
        input_notebook_file=input_notebook,
        output_notebook_folder=output_notebook_folder,
        parameters=f'PROJECT_ID={project_id},EXECUTION_ID={execution_id}'
    )

For additional details about:
- Google Cloud Notebook Executor template, see the [ExecutionTemplate API documentation](https://cloud.google.com/notebooks/docs/reference/rest/v1/ExecutionTemplate).
- Pipeline types, see [kfp.dsl.types](https://kubeflow-pipelines.readthedocs.io/en/latest/_modules/kfp/dsl/types.html).
- Notebook parameters, see [Papermill parameters](https://papermill.readthedocs.io/en/latest/usage-parameterize.html)

## Run pipeline

In [8]:
NOW = datetime.now().strftime("%Y%m%d%H%M%S")
JOB_ID=f'naas-{NOW}'
EXECUTION_ID=f'naas_{NOW}'

compiler.Compiler().compile(
    pipeline_func=pipeline, package_path=PACKAGE_PATH)

pl = pipeline_jobs.PipelineJob(
    display_name='notebook-executor-pipeline',
    template_path=PACKAGE_PATH,
    job_id=JOB_ID,
    parameter_values={
        'project_id': PROJECT_ID,
        'execution_id': EXECUTION_ID,
        'input_notebook': INPUT_NOTEBOOK_FILE,
        'output_notebook_folder': OUTPUT_NOTEBOOK_FOLDER
    },
)

pl.run()

INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/780283066030/locations/us-central1/pipelineJobs/naas-20210928144605
INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:
INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/780283066030/locations/us-central1/pipelineJobs/naas-20210928144605')
INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/naas-20210928144605?project=780283066030
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/780283066030/locations/us-central1/pipelineJobs/naas-20210928144605 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/780283066030/locations/us-central1/pipelineJobs/naas-20210928144605 current state:
PipelineState.

RuntimeError: Job failed with:
code: 9
message: "The DAG failed because some tasks failed. The failed tasks are: [execute-notebook].; Job (project_id = mam-nooage, job_id = 2102099106540290048) is failed due to the above error.; Failed to handle the job: {project_number = 780283066030, job_id = 2102099106540290048}"
